In [ ]:
try:
    # Check whether the c3 object is defined
    c3
except NameError:
    # Connect to a c3 cluster and create the c3 object
    from c3python import get_c3
    c3 = get_c3('<vanity_url>', '<tenant>', '<tag>')

In [57]:
#try:
#    # Check whether the c3 object is defined
#    c3
#except NameError:
    # Connect to a c3 cluster and create the c3 object
    from c3python import get_c3
    c3 = get_c3('http://192.168.2.44:8080', 'mnistExercise', 'dev')

Username: BA
Password: ········


In [67]:
!conda install -y dill

Solving environment: done

## Package Plan ##

  environment location: /data0/matthew/Software/NCSA/C3/c3-packages/dtiTraining/mnistExample/c3/py-tensorflow_2_1_0_venv

  added / updated specs:
    - dill


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    dill-0.3.2                 |     pyh9f0ad1d_0          59 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

  dill               conda-forge/noarch::dill-0.3.2-pyh9f0ad1d_0

The following packages will be UPDATED:

  openssl                                 1.1.1g-h516909a_0 --> 1.1.1g-h516909a_1



dill-0.3.2           | 59 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import dill

In [3]:
# Define function to create one-hot labels
def make_onehot(data):
    Res = np.zeros((data.size, data.max()+1))
    Res[np.arange(data.size), data.reshape((data.size,))] = 1
    return Res.reshape(data.shape+(data.max()+1,))

make_onehot_serialized = c3.PythonSerialization.serialize(make_onehot)

In [122]:
# Define function to scale data
def scale_data(data):
    return data/255.

scale_data_serialized = c3.PythonSerialization.serialize(scale_data)

In [123]:
mnist_test_images_spec = c3.IDXFile.get('MNIST_Test_Images').getFileSourceSpecPreprocess(scale_data_serialized, 'scale')
mnist_test_labels_spec = c3.IDXFile.get('MNIST_Test_Labels').getFileSourceSpecPreprocess(make_onehot_serialized, 'onehot')
mnist_train_images_spec = c3.IDXFile.get('MNIST_Train_Images').getFileSourceSpecPreprocess(scale_data_serialized, 'scale')
mnist_train_labels_spec = c3.IDXFile.get('MNIST_Train_Labels').getFileSourceSpecPreprocess(make_onehot_serialized, 'onehot')

In [124]:
# Cast file sources to datasets
mnist_test_images = c3.Dataset.fromNdArrayStreamable(sourceSpec=mnist_test_images_spec)
mnist_test_labels = c3.Dataset.fromNdArrayStreamable(sourceSpec=mnist_test_labels_spec)
mnist_train_images = c3.Dataset.fromNdArrayStreamable(sourceSpec=mnist_train_images_spec)
mnist_train_labels = c3.Dataset.fromNdArrayStreamable(sourceSpec=mnist_train_labels_spec)

In [115]:
mnist_test_images.toNumpyArray().shape

(7840000,)

In [110]:
test = mnist_test_images.toNumpyArray().data
print(test.shape)

(7840000,)


In [71]:
batchSpec = c3.BatchStreamSpec(batchSize=10)
stream_test = mnist_test_images.toStream(spec=batchSpec)

In [73]:
stream_test.next().data.shape

(10, 28, 28)

In [74]:
stream_test_2 = mnist_test_labels.toStream(spec=batchSpec)

In [75]:
stream_test_2.next().data.shape

(10, 10)

In [125]:
def train_and_test_model(name, model, train_X, train_Y, test_X, test_Y, epochs=10, callbacks=[]):
    # Create the keras pipe
    keras_pipe = c3.KerasPipe().upsertNativeModel(model=model)
    keras_pipe.technique.numEpochs = epochs

#    # Build the preprocessing step necessary to normalize the data.
#
#    scaler = c3.SklearnPipe(name="scaler",
#                            technique=c3.SklearnTechnique(
#                                #name="preprocessing.MinMaxScaler",
#                                name="preprocessing.StandardScaler",
#                                processingFunctionName="transform",
#                                )
#                            )
#
#    pipe = c3.MLSerialPipeline(name="serialPipeline",
#                               steps=[c3.MLStep(name="sklearnStep",
#                                                pipe=scaler),
#                                      c3.MLStep(name="kerasPipe",
#                                                pipe=keras_pipe)]
#                               )
    
#    trained_pipe = pipe.train(input=train_X, targetOutput=train_Y)
    trained_pipe = keras_pipe.train(input=train_X, targetOutput=train_Y)
    result = trained_pipe.process(input=test_X)

    result_np = result.toNumpyArray()
    test_Y_np = test_Y.toStream().collectAndMerge().data

    tot = test_Y_np.shape[0]
    error_rate = sum(result_np.argmax(axis=1)!=test_Y_np.argmax(axis=1))/(float(tot))

    print("{} Error Rate: {:.2f}%".format(name, error_rate*100))
    return trained_pipe

In [120]:
# Build tensorflow model.
one_layer_simple_model = tf.keras.Sequential([
    tf.keras.layers.Reshape((28*28,), input_shape=(28,28)),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

one_layer_simple_model.summary()
one_layer_simple_model.compile(optimizer='Adam', loss='categorical_crossentropy')

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_7 (Reshape)          (None, 784)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 1000)              785000    
_________________________________________________________________
dense_15 (Dense)             (None, 10)                10010     
Total params: 795,010
Trainable params: 795,010
Non-trainable params: 0
_________________________________________________________________


In [126]:
trained_pipe = train_and_test_model('one_layer_simple_model', one_layer_simple_model,
                     mnist_train_images, mnist_train_labels,
                     mnist_test_images, mnist_test_labels)

one_layer_simple_model Error Rate: 4.63%


In [103]:
# Build tensorflow model.
two_layer_simple_model = tf.keras.Sequential([
    tf.keras.layers.Reshape((28*28,), input_shape=(28,28)),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

two_layer_simple_model.summary()
two_layer_simple_model.compile(optimizer='Adam', loss='categorical_crossentropy')

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_3 (Reshape)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dense_7 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_8 (Dense)              (None, 10)                10010     
Total params: 1,796,010
Trainable params: 1,796,010
Non-trainable params: 0
_________________________________________________________________


In [127]:
train_and_test_model('two_layer_simple_model', two_layer_simple_model,
                     mnist_train_images, mnist_train_labels,
                     mnist_test_images, mnist_test_labels)

two_layer_simple_model Error Rate: 5.35%


c3.KerasPipe(
 id='868531ab-9312-4822-a0a1-7c8f95c8c6fc',
 meta=c3.Meta(
        tenantTagId=3,
        tenant='mnistExercise',
        tag='dev',
        created=datetime.datetime(2020, 8, 12, 21, 27, 52, tzinfo=datetime.timezone.utc),
        createdBy='BA',
        updated=datetime.datetime(2020, 8, 12, 21, 28, 9, tzinfo=datetime.timezone.utc),
        updatedBy='BA',
        timestamp=datetime.datetime(2020, 8, 12, 21, 28, 9, tzinfo=datetime.timezone.utc),
        fetchInclude='[]',
        fetchType='KerasPipe'),
 version=2,
 typeIdent='PIPE:LF:KRS',
 implType='KerasPipeV7d12',
 noTrainScore=False,
 persistedModelCategory='unidentified',
 typeVersion='7.12.4',
 untrainableOverride=False,
 technique=c3.KerasTechnique(
             modelDef='{"class_name": "Sequential", "config": {"name": '
                       '"sequential_3", "layers": [{"class_name": "Reshape", '
                       '"config": {"name": "reshape_3", "trainable": true, '
                       '"batch_input_sh

In [106]:
one_layer_cnn_model = tf.keras.Sequential([
    tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
    tf.keras.layers.Conv2D(32, kernel_size=(3,3),
                           activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

one_layer_cnn_model.summary()
one_layer_cnn_model.compile(optimizer='Adam', loss='categorical_crossentropy')

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_4 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
flatten (Flatten)            (None, 21632)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                216330    
Total params: 216,650
Trainable params: 216,650
Non-trainable params: 0
_________________________________________________________________


In [128]:
train_and_test_model('one_layer_cnn_model', one_layer_cnn_model,
                     mnist_train_images, mnist_train_labels,
                     mnist_test_images, mnist_test_labels)

one_layer_cnn_model Error Rate: 3.29%


c3.KerasPipe(
 id='be7ca671-6abb-4a1f-864e-74c15e4763a7',
 meta=c3.Meta(
        tenantTagId=3,
        tenant='mnistExercise',
        tag='dev',
        created=datetime.datetime(2020, 8, 12, 21, 28, 18, tzinfo=datetime.timezone.utc),
        createdBy='BA',
        updated=datetime.datetime(2020, 8, 12, 21, 28, 26, tzinfo=datetime.timezone.utc),
        updatedBy='BA',
        timestamp=datetime.datetime(2020, 8, 12, 21, 28, 26, tzinfo=datetime.timezone.utc),
        fetchInclude='[]',
        fetchType='KerasPipe'),
 version=2,
 typeIdent='PIPE:LF:KRS',
 implType='KerasPipeV7d12',
 noTrainScore=False,
 persistedModelCategory='unidentified',
 typeVersion='7.12.4',
 untrainableOverride=False,
 technique=c3.KerasTechnique(
             modelDef='{"class_name": "Sequential", "config": {"name": '
                       '"sequential_4", "layers": [{"class_name": "Reshape", '
                       '"config": {"name": "reshape_4", "trainable": true, '
                       '"batch_input_

In [108]:
three_layer_cnn_model = tf.keras.Sequential([
    tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
    tf.keras.layers.Conv2D(32, kernel_size=(3,3),
                           activation='relu'),
    tf.keras.layers.Conv2D(64, kernel_size=(3,3),
                           activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(125, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

three_layer_cnn_model.summary()
three_layer_cnn_model.compile(optimizer='Adam', loss='categorical_crossentropy')

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_5 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 125)              

In [130]:
train_and_test_model('three_layer_cnn_model', three_layer_cnn_model,
                     mnist_train_images, mnist_train_labels,
                     mnist_test_images, mnist_test_labels)

three_layer_cnn_model Error Rate: 2.07%


c3.KerasPipe(
 id='3165af73-aa06-47ff-bfcd-016b5c9e9915',
 meta=c3.Meta(
        tenantTagId=3,
        tenant='mnistExercise',
        tag='dev',
        created=datetime.datetime(2020, 8, 12, 21, 34, 19, tzinfo=datetime.timezone.utc),
        createdBy='BA',
        updated=datetime.datetime(2020, 8, 12, 21, 34, 46, tzinfo=datetime.timezone.utc),
        updatedBy='BA',
        timestamp=datetime.datetime(2020, 8, 12, 21, 34, 46, tzinfo=datetime.timezone.utc),
        fetchInclude='[]',
        fetchType='KerasPipe'),
 version=2,
 typeIdent='PIPE:LF:KRS',
 implType='KerasPipeV7d12',
 noTrainScore=False,
 persistedModelCategory='unidentified',
 typeVersion='7.12.4',
 untrainableOverride=False,
 technique=c3.KerasTechnique(
             modelDef='{"class_name": "Sequential", "config": {"name": '
                       '"sequential_5", "layers": [{"class_name": "Reshape", '
                       '"config": {"name": "reshape_5", "trainable": true, '
                       '"batch_input_